In [39]:
import pandas as pd
%matplotlib inline

# Assigment 4 - Ford GoBike by Steve Dille

Problem Statement
- You're a data scientist at Ford GoBike (https://www.fordgobike.com/), the company running Bay Area Bikeshare. You are trying to increase ridership, and you want to offer deals through the mobile app to do so. What deals do you offer 
though? Currently, your company has three options: a flat price for a single one-way trip, a day pass that allows 
unlimited 30-minute rides for 24 hours and an annual membership. 

- Through this project, you will answer these questions: 
  * What are the 5 most popular trips that you would call "commuter trips"?
  * What are your recommendations for offers (justify based on your findings)?
    
Assignment 04 - Employ notebooks to synthesize query project results
* Get Going
* Use JupyterHub on your midsw205 droplet to create a new python3 notebook.

Run queries in the notebook
! bq query --use_legacy_sql=FALSE '<your-query-here>'

Report
- Short description of findings and recommendations
- If needed, add data visualizations to support recommendations

# Analysis and Strategies for Increasing Riders and Revenues for Ford Bikes

The general strategy we will employ is to increase the utilization of bike day passes to increase revenue.  To accomplish this, we analyze the data and look for low periods of ridership relative to norms that would be indicitive of surplus bike availability.  We then propose promotions to increase the number of day passes and revenue during these periods. We do not want to offer promotions during periods of peak ridership that would canabilize full price revenue streams.  

# Background Information Queries

What time period do we have trip data for?  We need to be sure data is consistent and the most recent as stations continued to be added or we may get misleading results. The most recent full year of data that would cover all holidays is 2015 so that will be used as 2016 is partial year.  

In [44]:
! bq query --use_legacy_sql=false 'SELECT min(start_date) as earliest_date, max(start_date) as latest_date FROM `bigquery-public-data.san_francisco.bikeshare_trips`'

Waiting on bqjob_r4454fb82b24c9707_00000168b57e1003_1 ... (0s) Current status: DONE   
+---------------------+---------------------+
|    earliest_date    |     latest_date     |
+---------------------+---------------------+
| 2013-08-29 09:08:00 | 2016-08-31 23:32:00 |
+---------------------+---------------------+


In [32]:
# What are the five most popular commuter trips?  I defined commuter trips to be: less than 30 minutes, between the 
# hours of 7AM to 10AM and 5PM to 8PM, being on Monday to Friday and not starting and ending at the same station as
# that is more likely an errand.  
# We probably want to avoid promotions during the most popular trip times.

! bq query --use_legacy_sql=false 'SELECT start_station_name, end_station_name, count(*) as num_trips FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE start_station_name <> end_station_name and duration_sec < 1800 and ((EXTRACT(hour from start_date) between 7 and 10) OR (EXTRACT(hour from start_date) between 17 and 20)) AND (EXTRACT (dayofweek from start_date) between 2 and 6) and subscriber_type = "Subscriber" and (EXTRACT (dayofweek from start_date) between 2 and 6) GROUP BY start_station_name, end_station_name ORDER BY num_trips desc limit 5'


Waiting on bqjob_r793ca15ca2ae5796_00000168b552f2b3_1 ... (1s) Current status: DONE   
+------------------------------------------+-----------------------------------------+-----------+
|            start_station_name            |            end_station_name             | num_trips |
+------------------------------------------+-----------------------------------------+-----------+
| San Francisco Caltrain 2 (330 Townsend)  | Townsend at 7th                         |      6249 |
| Harry Bridges Plaza (Ferry Building)     | 2nd at Townsend                         |      5151 |
| Steuart at Market                        | 2nd at Townsend                         |      4736 |
| San Francisco Caltrain (Townsend at 4th) | Harry Bridges Plaza (Ferry Building)    |      4332 |
| Townsend at 7th                          | San Francisco Caltrain 2 (330 Townsend) |      4191 |
+------------------------------------------+-----------------------------------------+-----------+


In [40]:
# Using Pandas and CSV
# What are the five most popular commuter trips?  
# We probably want to avoid promotions during the most popular trip times.
# Using CSV to Pandas for print formatting for the same query as above

! bq query --use_legacy_sql=false --format=csv 'SELECT start_station_name, end_station_name, count(*) as num_trips FROM `bigquery-public-data.san_francisco.bikeshare_trips` WHERE start_station_name <> end_station_name and duration_sec < 1800 and ((EXTRACT(hour from start_date) between 7 and 10) OR (EXTRACT(hour from start_date) between 17 and 20)) AND (EXTRACT (dayofweek from start_date) between 2 and 6) and subscriber_type = "Subscriber" and (EXTRACT (dayofweek from start_date) between 2 and 6) GROUP BY start_station_name, end_station_name ORDER BY num_trips desc limit 5' > result.csv


Waiting on bqjob_r7e4d9b990f122d6d_00000168b573e3d3_1 ... (0s) Current status: DONE   

In [41]:
pop_trips=pd.read_csv('result.csv')

In [42]:
pop_trips

,start_station_name,end_station_name,num_trips
0,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th,6249
1,Harry Bridges Plaza (Ferry Building),2nd at Townsend,5151
2,Steuart at Market,2nd at Townsend,4736
3,San Francisco Caltrain (Townsend at 4th),Harry Bridges Plaza (Ferry Building),4332
4,Townsend at 7th,San Francisco Caltrain 2 (330 Townsend),4191


In [51]:
# How many bikes are there in San Francisco vs. the south bay/penninsula?
# This information may be useful for ramping up promotions by region in the future.

! bq query --use_legacy_sql=false 'SELECT landmark, sum(dockcount) as Dock_Count FROM `bigquery-public-data.san_francisco.bikeshare_stations` GROUP BY landmark ORDER BY Dock_Count desc'

Waiting on bqjob_r4c52cc6fc919661f_00000168b594d8e2_1 ... (0s) Current status: DONE   
+---------------+------------+
|   landmark    | Dock_Count |
+---------------+------------+
| San Francisco |        735 |
| San Jose      |        302 |
| Mountain View |        117 |
| Redwood City  |        115 |
| Palo Alto     |         75 |
+---------------+------------+


# The Number of Weekend Bike Trips Are Far Lower Than Weekdays Indicating Unused Bicycles 

In [1]:
! bq query --use_legacy_sql=false 'SELECT round(count(*)/52) as Num_Weekly_Trips, CASE WHEN extract(DAYOFWEEK from start_date) = 1 THEN "Sunday" WHEN extract(DAYOFWEEK from start_date) = 2 THEN "Monday" WHEN extract(DAYOFWEEK from start_date) = 3 THEN "Tuesday" WHEN extract(DAYOFWEEK from start_date) = 4 THEN "Wednesday" WHEN extract(DAYOFWEEK from start_date) = 5 THEN "Thursday" WHEN extract(DAYOFWEEK from start_date) = 6 THEN "Friday" WHEN extract(DAYOFWEEK from start_date) = 7 THEN "Saturday" END Day_of_Week FROM `bigquery-public-data.san_francisco.bikeshare_trips`  WHERE date(start_date) = date(end_date) and EXTRACT(YEAR from start_date) = 2015  GROUP BY Day_of_Week ORDER BY Num_Weekly_Trips' 

Waiting on bqjob_r63c094ab3a18215e_00000168c454c30e_1 ... (1s) Current status: DONE   
+------------------+-------------+
| Num_Weekly_Trips | Day_of_Week |
+------------------+-------------+
|            290.0 | Sunday      |
|            363.0 | Saturday    |
|           1104.0 | Friday      |
|           1144.0 | Monday      |
|           1222.0 | Thursday    |
|           1244.0 | Wednesday   |
|           1277.0 | Tuesday     |
+------------------+-------------+


# The Economics of Weekend Trips - Subscribers vs. Paid & Trip Duration

In [3]:
! bq query --use_legacy_sql=false 'SELECT subscriber_type, round(count(*)/52) as avg_weekend_trips, round(avg(duration_sec/60)) as avg_ride_minutes  FROM `bigquery-public-data.san_francisco.bikeshare_trips`  WHERE date(start_date) = date(end_date) and EXTRACT(YEAR from start_date) = 2015 and (extract(DAYOFWEEK from start_date) = 1 or extract(DAYOFWEEK from start_date) = 7) group by subscriber_type'

Waiting on bqjob_r2d24227c9b4cca7c_00000168bbb60770_1 ... (1s) Current status: DONE   
+-----------------+-------------------+------------------+
| subscriber_type | avg_weekend_trips | avg_ride_minutes |
+-----------------+-------------------+------------------+
| Customer        |             310.0 |             55.0 |
| Subscriber      |             343.0 |             10.0 |
+-----------------+-------------------+------------------+


The above query helps us understand that there is under utilized supply of bikes on the weekends. In addition, the
majority of weekend trips are by subscribers presenting us with less cannibalization risk in establishing promotions that would reduce revenue from individual full price paid trips.

# Weekend Promotion Possibilities

Weekends are a time when visitors to the bay area and locals alike are interested in leisurly exploring the city without the hurried pace of the work week. Using a bicyle to move about is and interesting and cost effective means of transportation. Research has also shown that weekends are a prime time for catching up on exercise providing an additional incentive to ride. Given weekend trip volumes are approximately 25-30% of weekday trips, Ford Bikes can afford to offer additional free hours of bicycle usage with paid hours which will increase revenue overall while incenting customers to utilize our open weekend bikes.  We propose two promotion ideas through the mobile app.

1. For non-subscriber customers who occasionally make use of bikes on day passes or per trip passes, we can offer on weekend days: "2 hours for \$5".  This will allow riders to take a ride to the bridge or ride to breakfast, eat and 
return home for a flat \$5 fee which could not have been accomplished with our normal 30 minute trip limit pricing without overage charges. Overages beyond 2 hours are simply charged at the normal \$3 per 15 minute 
rate.  

To understand cannibalization risk, we recognize from the data above, that the majority of weekend trips (343) are made by subscribers who would pay Ford nothing additional anyway.  On average 310 trips are made on weekend days with an average duration of 55 minutes before bike check in. We don't know how many of these trips were made by day pass buyers who paid \$10 vs. single riders who would have paid \$8 for this amount of time. It seems likely, that given how close the average time is to one hour where additional minutes would exceed the cost of a day pass, that most trips are on a day pass. Therefore, we believe that the \$5 for 2 hour offer will incent new trial ridership without cannibilizing the \$10 day pass revenue we currently recognize.

2. For new customers, we can offer "Bike it Off" a way to promote using a bike for exercise on the weekend. This promo has identical terms to "2 Hours for $5", but in addition to selling unused capacity, we are attempting to allow customers of this offer to sample what it is like to use our bikes, with the hope that they will become a more regular customer.

The mobile app will allow us to distinguish these offers for the two segments of customers.

This analysis was based on the most recent year where full year data was available (2015) for a consistent monthly data set over a one year period.


# Database Schema Information

## Database Information
### bikeshare_trips
| Field name         | Type      | Mode     | Description                                                                                                  |
|--------------------|-----------|----------|--------------------------------------------------------------------------------------------------------------|
| trip_id            | INTEGER   | REQUIRED | Numeric ID of bike trip                                                                                      |
| duration_sec       | INTEGER   | NULLABLE | Time of trip in seconds                                                                                      |
| start_date         | TIMESTAMP | NULLABLE | Start date of trip with date and time, in PST                                                                |
| start_station_name | STRING    | NULLABLE | Station name of start station                                                                                |
| start_station_id   | INTEGER   | NULLABLE | Numeric reference for start station                                                                          |
| end_date           | TIMESTAMP | NULLABLE | End date of trip with date and time, in PST                                                                  |
| end_station_name   | STRING    | NULLABLE | Station name for end station                                                                                 |
| end_station_id     | INTEGER   | NULLABLE | Numeric reference for end station                                                                            |
| bike_number        | INTEGER   | NULLABLE | ID of bike used                                                                                              |
| zip_code           | STRING    | NULLABLE | Home zip code of subscriber (customers can choose to manually enter zip at kiosk however data is unreliable) |
| subscriber_type    | STRING    | NULLABLE | Subscriber = annual or 30-day member; Customer = 24-hour or 3-day member                                     |

### bikeshare_status
| Field name      | Type      | Mode     | Description               |   |
|-----------------|-----------|----------|---------------------------|---|
| station_id      | INTEGER   | REQUIRED | Station ID number         |   |
| bikes_available | INTEGER   | NULLABLE | Number of available bikes |   |
| docks_available | INTEGER   | NULLABLE | Number of available docks |   |
| time            | TIMESTAMP | NULLABLE | Date and time, PST        |   |

### bikeshare_stations
| Field name        | Type    | Mode     | Description                                                                        |   |
|-------------------|---------|----------|------------------------------------------------------------------------------------|---|
| station_id        | INTEGER | REQUIRED | Station ID number                                                                  |   |
| name              | STRING  | NULLABLE | Name of station                                                                    |   |
| latitude          | FLOAT   | NULLABLE | Latitude                                                                           |   |
| longitude         | FLOAT   | NULLABLE | Longitude                                                                          |   |
| dockcount         | INTEGER | NULLABLE | Number of total docks at station                                                   |   |
| landmark          | STRING  | NULLABLE | City (San Francisco, Redwood City, Palo Alto, Mountain View, San Jose)             |   |
| installation_date | DATE    | NULLABLE | Original date that station was installed. If station was moved, it is noted below. |   |